Import necessary libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import os


Mount the drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import the dataset and split it into training and validation set


In [ ]:
train_dir = '/content/drive/MyDrive/finaldataset/AtoZ_3.1'
val_dir = '/content/drive/MyDrive/finaldataset/AtoZ_3.1'

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% validation
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 3746 images belonging to 26 classes.
Found 935 images belonging to 26 classes.


Callout the Model

In [ ]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Modify the last layer of MobileNetV2


In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(26, activation='softmax')(x)  # 26 classes for each letter

model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
for layer in base_model.layers:
    layer.trainable = False


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


Train the model

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


117/117 ━━━━━━━━━━━━━━━━━━━━ 2468s 20s/step - accuracy: 0.6796 - loss: 1.1995 - val_accuracy: 0.9612 - val_loss: 0.1308
Epoch 2/10
  1/117 ━━━━━━━━━━━━━━━━━━━━ 4:34 2s/step - accuracy: 0.9062 - loss: 0.1870

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9062 - loss: 0.1870 - val_accuracy: 1.0000 - val_loss: 0.0239
Epoch 3/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 287s 2s/step - accuracy: 0.9670 - loss: 0.1446 - val_accuracy: 0.9569 - val_loss: 0.1317
Epoch 4/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 0.0623 - val_accuracy: 1.0000 - val_loss: 0.0038
Epoch 5/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 288s 2s/step - accuracy: 0.9658 - loss: 0.1090 - val_accuracy: 0.9655 - val_loss: 0.0947
Epoch 6/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9688 - loss: 0.0838 - val_accuracy: 0.8571 - val_loss: 0.4240
Epoch 7/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 286s 2s/step - accuracy: 0.9816 - loss: 0.0616 - val_accuracy: 0.9763 - val_loss: 0.0879
Epoch 8/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 1.0000 - loss: 0.0330 - val_accuracy: 1.0000 - val_loss: 0.0047
Epoch 9/10
117/117 ━━━━━━━━━━━━━━━━━━━━ 291s 2s/step - accuracy: 0.9817 - loss: 0.0643 - val_accuracy: 0.9795 - 

In [ ]:
model.save('/content/drive/MyDrive/your_model_folder/sign_language_model.h5')


In [ ]:
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy*100:.2f}%')


30/30 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.9732 - loss: 0.0851
Validation accuracy: 97.01%


In [ ]:
from tensorflow.keras.models import load_model

# Load the saved model
model = load_model('/content/drive/MyDrive/your_model_folder/sign_language_model.h5')


In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def prepare_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))  # Resize to match model input size
    img_array = image.img_to_array(img)  # Convert to array
    img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to match the input shape
    img_array = img_array / 255.0  # Normalize the image
    return img_array


In [ ]:
# Prepare the image
img_path = '/content/drive/MyDrive/images to test /WhatsApp Image 2024-08-31 at 1.08.31 AM.jpeg'  #image path
img_array = prepare_image(img_path)

# Make a prediction
predictions = model.predict(img_array)

# Get the class with the highest probability
predicted_class = np.argmax(predictions[0])


# List of class labels corresponding to the classes the model was trained on
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

predicted_label = class_labels[predicted_class]

print(f'The predicted sign is: {predicted_label}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
The predicted sign is: Q


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


test_dir = '/content/drive/MyDrive/finaldataset/AtoZ_3.1'  # path of dataset
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test dataset
loss, accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {accuracy * 100:.2f}%')


Found 4681 images belonging to 26 classes.
147/147 ━━━━━━━━━━━━━━━━━━━━ 251s 2s/step - accuracy: 0.9771 - loss: 0.0762
Test accuracy: 98.16%
